In [158]:
from dotenv import load_dotenv

load_dotenv() # load session

True

In [159]:
import re
from functools import reduce
from aocd import get_data, submit
day = 7
year = 2023

input = get_data(day=day, year=year)

# submit(answer=result, part="a", day=day, year=year)

In [160]:
text = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

In [161]:
from itertools import groupby

labels = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]
hand_types = ['five_of_a_kind', 'four_of_a_kind', 'full_house', 'three_of_a_kind', 'two_pair', 'one_pair', 'high_card']

def highest(label1, label2):
  return labels.index(label1) <= labels.index(label2)

def highest_cards(cards):
  return reduce(lambda card1, card2: card1 if highest(card1, card2) else card2, cards)

def rest(cards, card_removed):
  return sorted(list(cards - card_removed), key=lambda card: labels.index(card))

def read_hand_type(cards):
  three_same_label = None
  two_same_labels = []

  for label, number in sorted(cards.items(), key=lambda item: item[1], reverse=True):
    if number == 5:
      return ('five_of_a_kind', label)
    elif number == 4:
      return ('four_of_a_kind', label, rest(cards.keys(), {label}))
    elif number == 3:
      three_same_label = label
    elif number == 2:
      two_same_labels.append(label)
    
  if three_same_label and two_same_labels != []:
    return ('full_house', three_same_label, two_same_labels[0])
  elif three_same_label:
    return ('three_of_a_kind', three_same_label, rest(cards.keys(), {three_same_label}))
  elif two_same_labels != []:
    if len(two_same_labels) == 2:
      highest_pair = highest_cards(two_same_labels)
      lowest_pair = two_same_labels[1] if highest_pair == two_same_labels[0] else two_same_labels[0]
      return ('two_pair', highest_pair, lowest_pair, rest(cards.keys(), {lowest_pair, highest_pair}))
    else:
      return ('one_pair', two_same_labels[0], rest(cards.keys(), {two_same_labels[0]}))
  else:
    high_card = highest_cards(cards.keys())
    return ('high_card', high_card, rest(cards.keys(), {high_card}))

def read_hand(hand):
  cards = {label: len(list(cards))
          for label, cards in groupby(sorted(hand))}
  type = read_hand_type(cards)
  return (hand, type)

assert read_hand("32T3K") == ("32T3K", ('one_pair', '3', ['K', 'T', '2']))
assert read_hand("T55J5") == ("T55J5", ('three_of_a_kind', '5', ['J', 'T']))
assert read_hand("KK677") == ("KK677", ('two_pair', 'K', '7', ['6']))
assert read_hand("KTJJT") == ("KTJJT", ('two_pair', 'J', 'T', ['K']))
assert read_hand("QQQJA") == ("QQQJA", ('three_of_a_kind', 'Q', ['A', 'J']))

assert read_hand("55555") == ("55555", ('five_of_a_kind', '5'))
assert read_hand("4444A") == ("4444A", ('four_of_a_kind', '4', ['A']))
assert read_hand("KKKQQ") == ("KKKQQ", ('full_house', 'K', 'Q'))
assert read_hand("AKQJT") == ("AKQJT", ('high_card', 'A', ['K', 'Q', 'J', 'T']))

In [162]:
def read_line(line):
  parts = line.split(" ")
  return (read_hand(parts[0]), int(parts[1]))

def read_hands_and_bids(text):
  return [read_line(line) for line in text.splitlines()]

example_hands_and_bids = read_hands_and_bids(text)

example_hands_and_bids

[(('32T3K', ('one_pair', '3', ['K', 'T', '2'])), 765),
 (('T55J5', ('three_of_a_kind', '5', ['J', 'T'])), 684),
 (('KK677', ('two_pair', 'K', '7', ['6'])), 28),
 (('KTJJT', ('two_pair', 'J', 'T', ['K'])), 220),
 (('QQQJA', ('three_of_a_kind', 'Q', ['A', 'J'])), 483)]

In [163]:
def left_wins(cards1, cards2):
  for i, _ in enumerate(cards1):
    if highest(cards1[i], cards2[i]):
      return True
    elif highest(cards2[i], cards1[i]):
      return False
  return False

def highest_hand(hand1, hand2):
  if hand1 == hand2:
    raise Exception("hands are equal")

  cards1, type1 = hand1
  cards2, type2 = hand2

  if type1[0] == type2[0]:
    type = type1[0]

    if type == 'five_of_a_kind':
      return hand1 if highest(type1[1], type2[1]) else hand2
    elif type == 'four_of_a_kind':
      if type1[1] != type2[1]:
        return hand1 if highest(type1[1], type2[1]) else hand2
      else:
        return hand1 if left_wins(type1[2], type2[2]) else hand2
    elif type == 'full_house':
      if type1[1] != type2[1]:
        return hand1 if highest(type1[1], type2[1]) else hand2
      else:
        return hand1 if highest(type1[2], type2[2]) else hand2
    elif type == 'three_of_a_kind':
      if type1[1] != type2[1]:
        return hand1 if highest(type1[1], type2[1]) else hand2
      else:
        return hand1 if left_wins(type1[2], type2[2]) else hand2
    elif type == "two_pair":
      if type1[1] != type2[1]:
        return hand1 if highest(type1[1], type2[1]) else hand2
      elif type1[2] != type2[2]:
        return hand1 if highest(type1[2], type2[2]) else hand2
      else:
        return hand1 if left_wins(type1[3], type2[3]) else hand2
    elif type == "one_pair":
      if type1[1] != type2[1]:
        return hand1 if highest(type1[1], type2[1]) else hand2
      else:
        return hand1 if left_wins(type1[2], type2[2]) else hand2
    elif type == "high_card":
      return hand1 if left_wins(type1[1], type2[1]) else hand2
  else:
    return hand1 if hand_types.index(type1[0]) < hand_types.index(type2[0]) else hand2

assert highest_hand(read_hand("KKKKK"), read_hand("34567")) == read_hand("KKKKK")
assert highest_hand(read_hand("KKKKK"), read_hand("QQQQQ")) == read_hand("KKKKK")
assert highest_hand(read_hand("KKKKK"), read_hand("QQQQJ")) == read_hand("KKKKK")
assert highest_hand(read_hand("KKKK2"), read_hand("QQQQJ")) == read_hand("KKKK2")
assert highest_hand(read_hand("KKKK2"), read_hand("KKKK3")) == read_hand("KKKK3")
assert highest_hand(read_hand("KKKA5"), read_hand("QQQA5")) == read_hand("KKKA5")
assert highest_hand(read_hand("KKK23"), read_hand("KKK43")) == read_hand("KKK43")
assert highest_hand(read_hand("88552"), read_hand("22334")) == read_hand("88552")
assert highest_hand(read_hand("33552"), read_hand("22334")) == read_hand("33552")
assert highest_hand(read_hand("55442"), read_hand("5544A")) == read_hand("5544A")
assert highest_hand(read_hand("33QKJ"), read_hand("22QKJ")) == read_hand("33QKJ")
assert highest_hand(read_hand("33QKJ"), read_hand("33A52")) == read_hand("33A52")
assert highest_hand(read_hand("23456"), read_hand("34567")) == read_hand("34567")

In [178]:
def left_higher_than(hand1, hand2):
  return highest_hand(hand1, hand2) == hand1

assert left_higher_than(read_hand("KKKKK"), read_hand("34567")) == True

In [186]:
from functools import cmp_to_key

def compare_hand_and_bid(hand_and_bid1, hand_and_bid2):
  if sorted(hand_and_bid1[0][0]) == sorted(hand_and_bid2[0][0]):
    return 0
  if left_higher_than(hand_and_bid1[0], hand_and_bid2[0]):
    return -1
  else:
    return 1

example_hands_and_bids.sort(key=cmp_to_key(compare_hand_and_bid), reverse=True)

winnings = [(rank * bid) for rank, (hand, bid) in enumerate(example_hands_and_bids, 1)]

assert sum(winnings) == 6440

In [192]:
hands_and_bids = read_hands_and_bids(input)

example_hands_and_bids.sort(key=cmp_to_key(compare_hand_and_bid), reverse=True)

winnings = [(rank * bid) for rank, (hand, bid) in enumerate(hands_and_bids, 1)]

result = sum(winnings)

submit(answer=result, part="a", day=day, year=year)

wrong answer: That's not the right answer; your answer is too low.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. [Return to Day 7]


That's not the right answer; your answer is too low.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. [Return to Day 7]
